<a href="https://colab.research.google.com/github/JuanSpecht/PDI2021/blob/main/TP5/PDI_TP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajo Práctico 5: Transformada de Fourier y procesamiento espectral

### Importo bibliotecas

In [2]:
import numpy as np
from google.colab import files
from PIL import Image
from io import BytesIO
import ipywidgets as widgets
import IPython.display as ipd

### Código relativo a la interfaz gráfica

In [5]:
class ModelUI:

    def __init__(self):

        self.par = dict()
        self.initUI()

    def initUI(self):

        # Inicializo las funciones que manejan los widgets

        self.initLoadImgButton()
        self.initSaveImgButton()
        self.initLoadSpectrumButton()
        self.initSaveSpectrumButton()
        self.initFTButton()
        self.initIFTButton()
        self.initImageView()
        self.initSpectrumView()

        # Layout de la interfaz

        self.image_panel = widgets.VBox([
            widgets.HBox([self.load_img_button,
                          self.save_img_button]),
            self.image_view
            ],
            layout=widgets.Layout(width='330px')
            )
        
        self.controls = widgets.VBox([
            self.ft_button,
            self.ift_button
            ],
            layout=widgets.Layout(width='200px',align_items='stretch', justify_content= 'center')
            )
        
        self.spectrum_panel = widgets.VBox([
            widgets.HBox([self.load_spectrum_button,
                          self.save_spectrum_button]),
            self.spectrum_view
            ],
            layout=widgets.Layout(width='330px')
            )
        
        self.UI = widgets.HBox([
            self.image_panel,
            self.controls,
            self.spectrum_panel
            ],
            layout=widgets.Layout(height='360px')
            )

    # Funciones que definen el comportamiento de los widgets

    def initImageView(self):
        self.image_view = widgets.Box(layout = widgets.Layout(
            overflow_x='hidden',
            overflow_y='hidden',
            width='320px',
            height='320px',
            justify_content='center',
            align_items='center'
            )
        )
    
    def initLoadImgButton(self):
        self.load_img_button = widgets.FileUpload(
            accept='',
            multiple=False,
            description="Cargar imagen"
            )
        
        self.load_img_button.observe(self.loadImage, "value")

    def loadImage (self, change):
        img = np.array(Image.open(BytesIO(self.load_img_button.data[0])).convert('L'))

        self.par["image"] = img

        # Paso la imagen a bytes
        img_in = Image.fromarray(np.uint8(self.par["image"]))
        buf = BytesIO()
        img_in.save(buf, format='JPEG')
        bytes_img_in = buf.getvalue()
        
        self.image_view.children = [widgets.Image(
            value=bytes_img_in,
            format='jpg',
            layout = widgets.Layout(
                object_fit='contain',
                overflow_x='hidden',
                overflow_y='hidden'
                )
            )
        ]

    def initSpectrumView(self):
        self.spectrum_view = widgets.Box(layout = widgets.Layout(
            overflow_x='hidden',
            overflow_y='hidden',
            width='320px',
            height='320px',
            justify_content='center',
            align_items='center'
            )
        )
    
    def initLoadSpectrumButton(self):
        self.load_spectrum_button = widgets.FileUpload(
            accept='',
            multiple=False,
            description="Cargar espectro"
            )
        
        self.load_spectrum_button.observe(self.loadSpectrum, "value")

    def loadSpectrum(self, change):
        try:
            phase = np.genfromtxt('/content/phase.csv', delimiter=',')
            mag_min_max = np.genfromtxt('/content/mag_min_max.csv', delimiter=',')

            self.par["phase"] = phase
            self.par["mag_min_max"] = mag_min_max
        except:
            pass
            
        spectrum = np.array(Image.open(BytesIO(self.load_spectrum_button.data[0])).convert('L'))
        self.par["spectrum"] = spectrum

        # Paso la imagen a bytes
        img = Image.fromarray(np.uint8(self.par["spectrum"]))
        buf = BytesIO()
        img.save(buf, format='JPEG')
        bytes_img = buf.getvalue()

        self.spectrum_view.children = [widgets.Image(
            value=bytes_img,
            format='jpg',
            layout = widgets.Layout(object_fit = 'contain',
                                    overflow_x='hidden',
                                    overflow_y='hidden'
                                    )
            )]

    def initFTButton(self):
        self.ft_button = widgets.Button(description="Aplicar TF >")

        self.ft_button.on_click(self.applyFT)

    def applyFT(self, change):
        try:
            fc = np.fft.fftshift(np.fft.fft2(self.par["image"]))
            mag = np.abs(fc)
            phase = np.angle(fc)
            mag_min = np.log(mag).min()
            mag_max = np.log(mag).max()
            mag_min_max = [mag_min, mag_max]

            '''
            En la magnitud del espectro resultante de la FFT la inmensa mayoría de
            los valores son muy pequeños. Para visualizarlo correctamente tomo el 
            logaritmo y mapeo todos los valores al intervalo [0,255]
            '''
            spectrum_img = np.interp(np.log(mag), (np.log(mag).min(), np.log(mag).max()), (0, 255))

            self.par["mag"] = mag
            self.par["phase"] = phase
            self.par["mag_min_max"] = mag_min_max
            self.par["spectrum"] = spectrum_img
            
        

            # Paso la imagen a bytes
            img_out = Image.fromarray(np.uint8(self.par["spectrum"]), 'L')
            buf = BytesIO()
            img_out.save(buf, format='PNG')
            bytes_img = buf.getvalue()

            self.spectrum_view.children = [widgets.Image(
                    value=bytes_img,
                    format='png',
                    layout = widgets.Layout(object_fit='contain',
                                            overflow_x='hidden',
                                            overflow_y='hidden'
                                            )
                    )]
        except:
            pass

    def initIFTButton(self):
        self.ift_button = widgets.Button(description="< Aplicar IFT")

        self.ift_button.on_click(self.applyIFT)

    def applyIFT(self, change):
        try:
            mag_min = self.par["mag_min_max"][0]
            mag_max = self.par["mag_min_max"][1]
            mag_norm = self.par["spectrum"]
            mag = np.exp(np.interp(mag_norm, (0, 255), (mag_min, mag_max)))
            spectrum = mag * (np.cos(self.par["phase"]) + np.sin(self.par["phase"])*1j)
            img_ift = np.clip(np.abs(np.fft.ifft2(spectrum)), 0, 255)

            self.par["image"] = img_ift

            # Paso la imagen a bytes
            img_ift1 = Image.fromarray(np.uint8(self.par["image"]), 'L')
            buf = BytesIO()
            img_ift1.save(buf, format='JPEG')
            bytes_img = buf.getvalue()

            self.image_view.children = [widgets.Image(
                    value=bytes_img,
                    format='jpg',
                    layout = widgets.Layout(object_fit='contain',
                                            overflow_x='hidden',
                                            overflow_y='hidden'
                                            )
                    )]
        except:
            print("No existe información de la fase")
            pass
        
    def initSaveImgButton(self):
        self.save_img_button = widgets.Button(description="Guardar Imagen")

        self.save_img_button.on_click(self.saveImage)

    def saveImage(self,change):
        try:
            im = Image.fromarray(self.par["image"].astype(np.uint8))
            im.save('image.png')

            print('La imagen ha sido guardada como \'image.png\'')
        except:
            pass

    def initSaveSpectrumButton(self):
        self.save_spectrum_button = widgets.Button(description="Guardar Espectro")

        self.save_spectrum_button.on_click(self.saveSpectrum)

    def saveSpectrum(self,change):
        try:
            im = Image.fromarray(self.par["spectrum"].astype(np.uint8))
            im.save('spectrum.png')
            files.download("/content/spectrum.png")

            # Guardo la fase y los extremos de la magnitud en archivos .csv
            np.savetxt("phase.csv", self.par["phase"], delimiter=",")
            np.savetxt("mag_min_max.csv", par['mag_min_max'], delimiter=",")
            
            print('El espectro ha sido guardado como \'spectrum.png\'')
            print('La fase ha sido guardada como \'phase.csv\'')
        except:
            pass

    def _ipython_display_(self):
        display(self.UI)

ModelUI()

* Si se carga alguna de las imágenes `seno1.png`, `seno2.png` o `seno3.png` en el panel de la izquierda y se aplica la TF, se ve que aparecen dos líneas verticales y dos líneas horizontales. Esto se debe a que tomé el logaritmo para visualizar la transformada, con lo cual las componentes del espectro con menor intensidad son más visibles. De otro modo se deberían ver solamente los puntos donde se cruzan dichas líneas, que corresponden a la frecuencia del seno de cada imagen.

* Al cargar la imagen `imagen1.jpg` se ve que es una paisaje con una función senoidal superpuesta. En el espectro se ven las líneas caractrísticas de una función senoidal.

    Guardé el espectro en el archivo `espectro de imagen1.png` y lo retoqué para eliminar lo mejor posible las líneas correspondientes a la función senoidal. Este paso hubiera sido mucho más sencillo y con mejor resultado si no hubiera tomado el logaritmo de las magnitudes, ya que de esa manera las frecuencias correspondientes a la senoidal se verían acumuladas en dos puntos, y serían más sencillos de borrar.

    Al cargar la imagen `espectro de imagen1 retocado.png` en el panel de la derecha y aplicar la antitransformada, se ve que la función senoidal disminuyó pero no por completo.

    El archivo `espectro de imagen1 demasiado retocado.png` difiere ligeramente del anterior (retoqué un poco más las líneas correspondientes a la parte senoidal y modifiqué a propósito algunos píxeles de la parte central). Al cargarlo y aplicar la antitransformada se ve que se pierde información en la parte central de la imagen, esto muestra el grado de sensibilidad del proceso.

* Al cargar los círculos y los rectángulos se ve que para las figuras más grandes, el patrón resultante es más "apretado" que para las figuras más pequeñas. 

    Por otro lado, se puede notar que es lo mismo usar una imagen negra sobre fondo blanco que una imagen blanca sobre fondo negro, lo único importante es el salto de un valor a otro
